In [1]:
# Import Packages
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from datetime import datetime, date

In [2]:
# Change Directory
import os
os.chdir('swe_data/')
os.listdir()

['SN_SWE_WY1985.h5',
 'SN_SWE_WY1986.h5',
 'SN_SWE_WY1987.h5',
 'SN_SWE_WY1988.h5',
 'SN_SWE_WY1989.h5',
 'SN_SWE_WY1990.h5',
 'SN_SWE_WY1991.h5',
 'SN_SWE_WY1992.h5',
 'SN_SWE_WY1993.h5',
 'SN_SWE_WY1994.h5',
 'SN_SWE_WY1995.h5',
 'SN_SWE_WY1996.h5',
 'SN_SWE_WY1997.h5',
 'SN_SWE_WY1998.h5',
 'SN_SWE_WY1999.h5',
 'SN_SWE_WY2000.h5',
 'SN_SWE_WY2001.h5',
 'SN_SWE_WY2002.h5',
 'SN_SWE_WY2003.h5',
 'SN_SWE_WY2004.h5',
 'SN_SWE_WY2005.h5',
 'SN_SWE_WY2006.h5',
 'SN_SWE_WY2007.h5',
 'SN_SWE_WY2008.h5',
 'SN_SWE_WY2009.h5',
 'SN_SWE_WY2010.h5',
 'SN_SWE_WY2011.h5',
 'SN_SWE_WY2012.h5',
 'SN_SWE_WY2013.h5',
 'SN_SWE_WY2014.h5',
 'SN_SWE_WY2015.h5',
 'SN_SWE_WY2016.h5']

## Objective: Join Time Series Discharge & SWE
1. Test joining on single gage

2. Table Joining Pipeline on all gages

### Function Used in Later Code

In [3]:
### 2. Function for Extracting Index
def index_finder(lon,lat):
    # Longtitude finder
    if lon < -123.3 or lon > -117.6:
        print('Longitude of Input is out of range! lon:',lon)
        return None
    elif lat < 35.4 or lat > 42:
        print('Latitude of Input is out of range! lat:',lat)
    else: #longtitude and latitude are within reasonable range
        lon_idx = round((lon + 123.3) * 1000)
        lat_idx = round((lat - 35.4) * 1000)
    
        return int(lon_idx),int(lat_idx)

## 1. Test Joining on Single Gage

### Read Data

In [14]:
# Discharge Data
gage = pd.read_csv('../gage_discharge_lat_lon.csv')

# Create Features & assign initial values with -1
gage['swe_avg'] = -1
gage['swe_min'] = -1
gage['swe_max'] = -1

In [15]:
gage.head()

,time,ft,m3,gage,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_min,swe_max
0,1984-10-01,54.0,1.529110,11402000,-121.157675,39.855478,-120.690823,40.049659,-1,-1,-1
1,1984-10-02,52.0,1.472476,11402000,-121.157675,39.855478,-120.690823,40.049659,-1,-1,-1
2,1984-10-03,49.0,1.387525,11402000,-121.157675,39.855478,-120.690823,40.049659,-1,-1,-1
3,1984-10-04,49.0,1.387525,11402000,-121.157675,39.855478,-120.690823,40.049659,-1,-1,-1
4,1984-10-05,48.0,1.359209,11402000,-121.157675,39.855478,-120.690823,40.049659,-1,-1,-1


In [16]:
# Variable Defining
prev_year = 0

In [17]:
### 
for row_num in range(len(gage)):
    row_data = gage.iloc[row_num,:]
    row_time = row_data['time']
    if row_time[0:4] == '2015' and row_data['gage']==11189500:
        break

In [18]:
row_data

time       2015-01-01
ft               9.96
m3           0.282036
gage         11189500
ll_lon       -118.384
ll_lat        35.7286
tr_lon       -118.004
tr_lat        36.4378
swe_avg            -1
swe_min            -1
swe_max            -1
Name: 81905, dtype: object

In [19]:
### Extract Date from Gage Data to match SWE
date_format = "%Y-%m-%d"
d_date = datetime.strptime(row_time, date_format)

# Extract year of date
d_year = d_date.year
# Extract number of days from SWE Data
num_days = d_date- datetime.strptime('{}-1-1'.format(d_year),date_format)
num_days = num_days.days

print(f'Year: {d_year}  Day: {num_days}')


Year: 2015  Day: 0


In [20]:
## Obtain swe data
# if year of previous row does not match year of current row. Then Read Data
if prev_year != d_year: 
    swe = h5py.File(f'SN_SWE_WY{d_year}.h5', 'r')
    # # testing purpose
    # swe = h5py.File('/Users/apple/Desktop/UC_Berkeley/UCB_2022/w210/data/SN_SWE_WY1985.h5','r')
    lat = swe['lat'][0][::-1]
    lon = swe['lon'][:,0]
    lats,lons = np.meshgrid(lat,lon)
prev_year = d_year

swe_data = swe['SWE'][num_days]
# flip over yaxis as lats are in a descending order --> need to change to ascending order
swe_data_flip = swe_data[:,::-1]

#### Find SWE Interested Region with Lat Lon

In [21]:
### 4. Find closest idx to the lower left & upper right corner
ll_lon_idx,ll_lat_idx = index_finder(row_data['ll_lon'],row_data['ll_lat'])
tr_lon_idx,tr_lat_idx = index_finder(row_data['tr_lon'],row_data['tr_lat'])
region = swe_data_flip[ll_lon_idx:tr_lon_idx,ll_lat_idx:tr_lat_idx]

# # Get SWE values of surrounding gage regions
# lon_idx,lat_idx = index_finder(row_data['lon'],row_data['lat'])
# lon_idx,lat_idx = int(lon_idx),int(lat_idx)
# region = swe_data_flip[lon_idx-sur:lon_idx+sur,lat_idx-sur:lat_idx+sur]


In [22]:
# Print Shape of array
print('Region Shape:',region.shape)
# Print 
print('Unique Value:',np.unique(region,return_counts=True))
region

Region Shape: (380, 709)
Unique Value: (array([-32768,      0], dtype=int16), array([ 39327, 230093], dtype=int64))


array([[-32768, -32768, -32768, ...,      0,      0,      0],
       [-32768, -32768, -32768, ...,      0,      0,      0],
       [-32768, -32768, -32768, ...,      0,      0,      0],
       ...,
       [     0,      0,      0, ..., -32768, -32768, -32768],
       [     0,      0,      0, ..., -32768, -32768, -32768],
       [-32768, -32768,      0, ..., -32768, -32768, -32768]], dtype=int16)

In [ ]:
'''
 ---- OLD COORDINATES ---
Gage with swe values: 11266500 
Gage with no swe values: 11402000, 11318500,11208000
Gage with certain swe values: 11185500
Gage with limited swe values: 11189500, 11202710
Note: Gage 11202710 starts from 1988 while others start from 1985

 ---- SWE on NEW COORDINATES (10/15) ----
 11202710: 40,904 valid & 6616 n/a values from region of 198x240 array
 11266500: 126,687 valid & 2784 n/a values from region of 419x309 array
 11402000: 36,414 valid & 54,651 n/a values from region of 467x195 array
 11318500: 9510 valid & 29072 n/a values from region of 382x101 array
 11208000: 29,403 valid & 3461 n/a values from region of 208x158 array
 11185500: 348,392 valid & 21336 n/a values from region of 436x848 array
 11189500: 230,093 valid & 39327 n/a values from region of 380x709 array
 --------------------------------------------------
'''

### Obtain Region of Interest

In [ ]:
# change null values to null
region=region.astype('float')
region[region == -32768] = np.nan

region_avg = np.nanmean(region)

if region_avg == np.nan:
    gage.loc[row_num,'swe_avg'] = -1
else: # region avg is not null
    gage.loc[row_num,'swe_avg'] = region_avg

# Pipeline: Data Joining Between Gage Time Series Data & SWE Value
- matching swe data to the gage time series data based on lat & lon of the gage
- **main code**

In [32]:
class gage_swe_join():
    def __init__(self):
        self.prev_year = 0
        self.date_format =  "%Y-%m-%d"
        
    def extract_swe_day_and_data(self,row_time):
        '''
        Objective:
            1. Obtain the year of the interested date
            2. Obtain the day of the year
            3. Extracted SWE data of the same date from gage data
    
        '''
        ### Extract Date from Gage Data to match SWE
        d_date = datetime.strptime(row_time, self.date_format)
        # Extract year of date
        d_year = d_date.year
        # Extract number of days from SWE Data
        num_days = d_date- datetime.strptime('{}-1-1'.format(d_year),self.date_format)
        num_days = num_days.days

        ## Obtain swe data
        # if year of previous row does not match year of current row. Then Read Data
        if self.prev_year != d_year: 
            self.swe = h5py.File(f'SN_SWE_WY{d_year}.h5', 'r')
    #         lat = swe['lat'][0][::-1]
    #         lon = swe['lon'][:,0]
    #         lats,lons = np.meshgrid(lat,lon)
        self.prev_year = d_year

        # extract swe date
        swe_data = self.swe['SWE'][num_days]

        # flip over yaxis as lats are in a descending order --> need to change to ascending order
        swe_data_flip = swe_data[:,::-1]

        return swe_data_flip
    
    ### 2. Function for Extracting Index
    def index_finder(self,lon,lat):
        # Longtitude finder
        if lon < -123.3 or lon > -117.6:
            print('Longitude of Input is out of range! lon:',lon)
            return None
        elif lat < 35.4 or lat > 42:
            print('Latitude of Input is out of range! lat:',lat)
        else: #longtitude and latitude are within reasonable range
            lon_idx = round((lon + 123.3) * 1000)
            lat_idx = round((lat - 35.4) * 1000)

            return int(lon_idx),int(lat_idx)

    def pipeline(self):
        '''
            Objective: Pipeline for joining the entire gage time series table with swe data
            Input:
                - gage discharge data (date, discharge, gage lat &lon)
                - swe data (date, lat& lon, swe)
            Output:
                - Joint times series table with both gage discharge & swe
        '''
        # read gage csv file
        gage = pd.read_csv('../gage_discharge_lat_lon.csv')

        # Create Features & assign initial values with -1
        gage['swe_avg'] = -1
        # gage['swe_min'] = -1
        gage['swe_max'] = -1

        ### Run through all data
        for ii,row_num in enumerate(range(len(gage))):

            ### Start from row xx
            if ii >= 0:

                if ii % 100 == 0:
                    print(f'-------- Processing Row Number {ii} out of {len(gage)} ---------')

                ####
                # Start with each row
                row_data = gage.iloc[row_num,:]
                row_time = row_data['time']

                # NOTE: SWE has only data files from 1984 - 2016
                if 1984 < int(row_time[0:4]) <2017  : 

                    ### Obtain Value of Interested Region
                    ll_lon_idx,ll_lat_idx = self.index_finder(row_data['ll_lon'],row_data['ll_lat'])
                    tr_lon_idx,tr_lat_idx = self.index_finder(row_data['tr_lon'],row_data['tr_lat'])

                    # Obtain SWE data of Same Date
                    swe_data= self.extract_swe_day_and_data(row_time)

                    # get SWE values of surrounding region
                    region = swe_data[ll_lon_idx:tr_lon_idx,ll_lat_idx:tr_lat_idx]

                    # change -32768 (null values) to null
                    region=region.astype('float')
                    region[region == -32768] = np.nan

                    # --------------------------------------------------------------------
                    # Select way to process the region data into a single value or a k-dimension vector
                    region_avg = np.nanmean(region)
                    region_max = np.nanmax(region)
                    #########################

                   # region avg is not null, assign the value with average
                    gage.loc[row_num,'swe_avg'] = region_avg
                    gage.loc[row_num,'swe_max'] = region_max
                    
                # save data to csv file every 1000 rows processed
                if ii % 1000 == 0: 
                    gage.to_csv('../gage_with_swe.csv',index=False)
                    print(f'---- Round {ii} Save to CSV file')
                    
                    
        return gage


    def check_data(self):
        gage = pd.read_csv('../gage_with_swe.csv')
        
        print(gage.iloc[1000:1100,:])


In [33]:
# run class
gsj = gage_swe_join()
final = gsj.pipeline()
# gsj.check_data()

-------- Processing Row Number 0 out of 83274 ---------
---- Round 0 Save to CSV file
-------- Processing Row Number 100 out of 83274 ---------
-------- Processing Row Number 200 out of 83274 ---------
-------- Processing Row Number 300 out of 83274 ---------
-------- Processing Row Number 400 out of 83274 ---------
-------- Processing Row Number 500 out of 83274 ---------
-------- Processing Row Number 600 out of 83274 ---------
-------- Processing Row Number 700 out of 83274 ---------
-------- Processing Row Number 800 out of 83274 ---------
-------- Processing Row Number 900 out of 83274 ---------
-------- Processing Row Number 1000 out of 83274 ---------
---- Round 1000 Save to CSV file
-------- Processing Row Number 1100 out of 83274 ---------
-------- Processing Row Number 1200 out of 83274 ---------
-------- Processing Row Number 1300 out of 83274 ---------
-------- Processing Row Number 1400 out of 83274 ---------
-------- Processing Row Number 1500 out of 83274 ---------
-----

-------- Processing Row Number 13100 out of 83274 ---------
-------- Processing Row Number 13200 out of 83274 ---------
-------- Processing Row Number 13300 out of 83274 ---------
-------- Processing Row Number 13400 out of 83274 ---------
-------- Processing Row Number 13500 out of 83274 ---------
-------- Processing Row Number 13600 out of 83274 ---------
-------- Processing Row Number 13700 out of 83274 ---------
-------- Processing Row Number 13800 out of 83274 ---------
-------- Processing Row Number 13900 out of 83274 ---------
-------- Processing Row Number 14000 out of 83274 ---------
---- Round 14000 Save to CSV file
-------- Processing Row Number 14100 out of 83274 ---------
-------- Processing Row Number 14200 out of 83274 ---------
-------- Processing Row Number 14300 out of 83274 ---------
-------- Processing Row Number 14400 out of 83274 ---------
-------- Processing Row Number 14500 out of 83274 ---------
-------- Processing Row Number 14600 out of 83274 ---------
------

---- Round 26000 Save to CSV file
-------- Processing Row Number 26100 out of 83274 ---------
-------- Processing Row Number 26200 out of 83274 ---------
-------- Processing Row Number 26300 out of 83274 ---------
-------- Processing Row Number 26400 out of 83274 ---------
-------- Processing Row Number 26500 out of 83274 ---------
-------- Processing Row Number 26600 out of 83274 ---------
-------- Processing Row Number 26700 out of 83274 ---------
-------- Processing Row Number 26800 out of 83274 ---------
-------- Processing Row Number 26900 out of 83274 ---------
-------- Processing Row Number 27000 out of 83274 ---------
---- Round 27000 Save to CSV file
-------- Processing Row Number 27100 out of 83274 ---------
-------- Processing Row Number 27200 out of 83274 ---------
-------- Processing Row Number 27300 out of 83274 ---------
-------- Processing Row Number 27400 out of 83274 ---------
-------- Processing Row Number 27500 out of 83274 ---------
-------- Processing Row Number 2

---- Round 39000 Save to CSV file
-------- Processing Row Number 39100 out of 83274 ---------
-------- Processing Row Number 39200 out of 83274 ---------
-------- Processing Row Number 39300 out of 83274 ---------
-------- Processing Row Number 39400 out of 83274 ---------
-------- Processing Row Number 39500 out of 83274 ---------
-------- Processing Row Number 39600 out of 83274 ---------
-------- Processing Row Number 39700 out of 83274 ---------
-------- Processing Row Number 39800 out of 83274 ---------
-------- Processing Row Number 39900 out of 83274 ---------
-------- Processing Row Number 40000 out of 83274 ---------
---- Round 40000 Save to CSV file
-------- Processing Row Number 40100 out of 83274 ---------
-------- Processing Row Number 40200 out of 83274 ---------
-------- Processing Row Number 40300 out of 83274 ---------
-------- Processing Row Number 40400 out of 83274 ---------
-------- Processing Row Number 40500 out of 83274 ---------
-------- Processing Row Number 4

---- Round 52000 Save to CSV file
-------- Processing Row Number 52100 out of 83274 ---------
-------- Processing Row Number 52200 out of 83274 ---------
-------- Processing Row Number 52300 out of 83274 ---------
-------- Processing Row Number 52400 out of 83274 ---------
-------- Processing Row Number 52500 out of 83274 ---------
-------- Processing Row Number 52600 out of 83274 ---------
-------- Processing Row Number 52700 out of 83274 ---------
-------- Processing Row Number 52800 out of 83274 ---------
-------- Processing Row Number 52900 out of 83274 ---------
-------- Processing Row Number 53000 out of 83274 ---------
---- Round 53000 Save to CSV file
-------- Processing Row Number 53100 out of 83274 ---------
-------- Processing Row Number 53200 out of 83274 ---------
-------- Processing Row Number 53300 out of 83274 ---------
-------- Processing Row Number 53400 out of 83274 ---------
-------- Processing Row Number 53500 out of 83274 ---------
-------- Processing Row Number 5

---- Round 65000 Save to CSV file
-------- Processing Row Number 65100 out of 83274 ---------
-------- Processing Row Number 65200 out of 83274 ---------
-------- Processing Row Number 65300 out of 83274 ---------
-------- Processing Row Number 65400 out of 83274 ---------
-------- Processing Row Number 65500 out of 83274 ---------
-------- Processing Row Number 65600 out of 83274 ---------
-------- Processing Row Number 65700 out of 83274 ---------
-------- Processing Row Number 65800 out of 83274 ---------
-------- Processing Row Number 65900 out of 83274 ---------
-------- Processing Row Number 66000 out of 83274 ---------
---- Round 66000 Save to CSV file
-------- Processing Row Number 66100 out of 83274 ---------
-------- Processing Row Number 66200 out of 83274 ---------
-------- Processing Row Number 66300 out of 83274 ---------
-------- Processing Row Number 66400 out of 83274 ---------
-------- Processing Row Number 66500 out of 83274 ---------
-------- Processing Row Number 6

---- Round 78000 Save to CSV file
-------- Processing Row Number 78100 out of 83274 ---------
-------- Processing Row Number 78200 out of 83274 ---------
-------- Processing Row Number 78300 out of 83274 ---------
-------- Processing Row Number 78400 out of 83274 ---------
-------- Processing Row Number 78500 out of 83274 ---------
-------- Processing Row Number 78600 out of 83274 ---------
-------- Processing Row Number 78700 out of 83274 ---------
-------- Processing Row Number 78800 out of 83274 ---------
-------- Processing Row Number 78900 out of 83274 ---------
-------- Processing Row Number 79000 out of 83274 ---------
---- Round 79000 Save to CSV file
-------- Processing Row Number 79100 out of 83274 ---------
-------- Processing Row Number 79200 out of 83274 ---------
-------- Processing Row Number 79300 out of 83274 ---------
-------- Processing Row Number 79400 out of 83274 ---------
-------- Processing Row Number 79500 out of 83274 ---------
-------- Processing Row Number 7

In [31]:
final.iloc[140:190,:]

,time,ft,m3,gage,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_max
140,1985-02-18,191.0,5.408518,11402000,-121.157675,39.855478,-120.690823,40.049659,38.607514,449.0
141,1985-02-19,189.0,5.351884,11402000,-121.157675,39.855478,-120.690823,40.049659,39.922145,456.0
142,1985-02-20,190.0,5.380201,11402000,-121.157675,39.855478,-120.690823,40.049659,44.145301,474.0
143,1985-02-21,176.0,4.983765,11402000,-121.157675,39.855478,-120.690823,40.049659,51.167353,493.0
144,1985-02-22,172.0,4.870498,11402000,-121.157675,39.855478,-120.690823,40.049659,52.312490,494.0
145,1985-02-23,174.0,4.927131,11402000,-121.157675,39.855478,-120.690823,40.049659,54.186824,499.0
146,1985-02-24,176.0,4.983765,11402000,-121.157675,39.855478,-120.690823,40.049659,77.601087,581.0
147,1985-02-25,179.0,5.068716,11402000,-121.157675,39.855478,-120.690823,40.049659,87.283298,610.0
148,1985-02-26,177.0,5.012082,11402000,-121.157675,39.855478,-120.690823,40.049659,104.234827,645.0
149,1985-02-27,172.0,4.870498,11402000,-121.157675,39.855478,-120.690823,40.049659,130.844785,697.0


In [ ]:
#################################################################

In [34]:
gage = pd.read_csv('../gage_with_swe.csv')

In [36]:
gage.iloc[82019:82050,:]

,time,ft,m3,gage,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_max
82019,2015-04-25,11.20,0.317149,11189500,-118.383732,35.728555,-118.003533,36.437843,11.725215,281.0
82020,2015-04-26,11.40,0.322812,11189500,-118.383732,35.728555,-118.003533,36.437843,11.505743,281.0
82021,2015-04-27,11.40,0.322812,11189500,-118.383732,35.728555,-118.003533,36.437843,11.189471,281.0
82022,2015-04-28,11.00,0.311485,11189500,-118.383732,35.728555,-118.003533,36.437843,10.844889,281.0
82023,2015-04-29,10.60,0.300159,11189500,-118.383732,35.728555,-118.003533,36.437843,11.201375,283.0
82024,2015-04-30,9.87,0.279487,11189500,-118.383732,35.728555,-118.003533,36.437843,11.543606,288.0
82025,2015-05-01,9.34,0.264479,11189500,-118.383732,35.728555,-118.003533,36.437843,11.360711,288.0
82026,2015-05-02,8.91,0.252303,11189500,-118.383732,35.728555,-118.003533,36.437843,11.263433,287.0
82027,2015-05-03,8.81,0.249471,11189500,-118.383732,35.728555,-118.003533,36.437843,11.187707,288.0
82028,2015-05-04,8.53,0.241543,11189500,-118.383732,35.728555,-118.003533,36.437843,10.983268,287.0


In [42]:
def extract_swe_day_and_data(row_time,date_format,prev_year,swe):
    '''
        Objective:
            1. Obtain the year of the interested date
            2. Obtain the day of the year
            3. Extracted SWE data of the same date from gage data
    
    '''
    ### Extract Date from Gage Data to match SWE
    d_date = datetime.strptime(row_time, date_format)
    # Extract year of date
    d_year = d_date.year
    # Extract number of days from SWE Data
    num_days = d_date- datetime.strptime('{}-1-1'.format(d_year),date_format)
    num_days = num_days.days

    ## Obtain swe data
    # if year of previous row does not match year of current row. Then Read Data
    print(prev_year)
    if prev_year != d_year: 
        swe = h5py.File(f'SN_SWE_WY{d_year}.h5', 'r')
#         lat = swe['lat'][0][::-1]
#         lon = swe['lon'][:,0]
#         lats,lons = np.meshgrid(lat,lon)
    prev_year = d_year

    # extract swe date
    swe_data = swe['SWE'][num_days]
    
    # flip over yaxis as lats are in a descending order --> need to change to ascending order
    swe_data_flip = swe_data[:,::-1]

    return swe_data_flip,prev_year,swe


In [43]:
# gage = pd.read_csv('../gage_swe.csv')
date_format = "%Y-%m-%d"
index_list = []
prev_year = 0

### Run through all data
for ii,row_num in enumerate(range(len(gage))):
    
    ### Start from row xx
    if ii >=0:
    
        if ii % 100 == 0:
            print(f'-------- Processing Row Number {ii} out of {len(gage)} ---------')

        ####
        # Start with each row
        row_data = gage.iloc[row_num,:]
        row_time = row_data['time']

        # NOTE: SWE has only data files from 1984 - 2016
        if 1984 < int(row_time[0:4]) <2017  : 
            
            ### Obtain Value of Interested Region
            ll_lon_idx,ll_lat_idx = index_finder(row_data['ll_lon'],row_data['ll_lat'])
            tr_lon_idx,tr_lat_idx = index_finder(row_data['tr_lon'],row_data['tr_lat'])
            
            # Obtain SWE data of Same Date
            swe_data,prev_year,swe = extract_swe_day_and_data(row_time,date_format,prev_year,swe)

            # get SWE values of surrounding region
            region = swe_data[ll_lon_idx:tr_lon_idx,ll_lat_idx:tr_lat_idx]

            # change -32768 (null values) to null
            region=region.astype('float')
            region[region == -32768] = np.nan

            ### Select way to process the region data into a single value or a k-dimension vector
            region_avg = np.nanmean(region)
            #########################

           # region avg is not null, assign the value with average
            gage.loc[row_num,'swe_avg'] = region_avg
    
        if ii % 1000 == 0: # save data to csv file every 1000 rows processed
            gage.to_csv('../gage_with_swe.csv',index=False)
            print(f'---- Round {ii} Save to CSV file')

-------- Processing Row Number 0 out of 83274 ---------
---- Round 0 Save to CSV file
0
before swe
1985
before swe


UnboundLocalError: local variable 'swe' referenced before assignment

In [28]:
# # save gage file to csv
# gage.to_csv('../gage_with_swe.csv',index=False)

In [67]:
gage.iloc[140:150,:]

,time,ft,m3,gage,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_min,swe_max
140,1985-02-18,191.0,5.408518,11402000,-121.157675,39.855478,-120.690823,40.049659,0.103175,-1,-1
141,1985-02-19,189.0,5.351884,11402000,-121.157675,39.855478,-120.690823,40.049659,0.241555,-1,-1
142,1985-02-20,190.0,5.380201,11402000,-121.157675,39.855478,-120.690823,40.049659,0.543417,-1,-1
143,1985-02-21,176.0,4.983765,11402000,-121.157675,39.855478,-120.690823,40.049659,0.622480,-1,-1
144,1985-02-22,172.0,4.870498,11402000,-121.157675,39.855478,-120.690823,40.049659,0.772478,-1,-1
145,1985-02-23,174.0,4.927131,11402000,-121.157675,39.855478,-120.690823,40.049659,0.657742,-1,-1
146,1985-02-24,176.0,4.983765,11402000,-121.157675,39.855478,-120.690823,40.049659,0.533586,-1,-1
147,1985-02-25,179.0,5.068716,11402000,-121.157675,39.855478,-120.690823,40.049659,0.401851,-1,-1
148,1985-02-26,177.0,5.012082,11402000,-121.157675,39.855478,-120.690823,40.049659,0.270445,-1,-1
149,1985-02-27,172.0,4.870498,11402000,-121.157675,39.855478,-120.690823,40.049659,0.182897,-1,-1


In [ ]:
'''
## SWE value indication
Initial Assigned Value: -1
N/A value: -2
Rest are normal values
'''

In [ ]:
# ---------------------------